In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

print(tf.__version__)
print(len(tf.config.experimental.list_physical_devices('GPU')))

2.7.0
1


In [ ]:
#IMPORTAÇÃO E NORRMALIZAÇÃO
(x_train, y_train), (x_test,y_test) = keras.datasets.cifar10.load_data()
#x_train = x_train.reshape(-1, 28, 28, 1)
#x_test = x_test.reshape(-1, 28, 28, 1)

x_train = x_train.astype(float)/255
x_test = x_test.astype(float)/255

#CRIAR DATASET
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(50000).batch(64)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(64)

170508288/170498071 [==============================] - 11s 0us/step


In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)
acc_metric = keras.metrics.SparseCategoricalAccuracy()
test_accuracy = tf.keras.metrics.Accuracy()
test_loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
train_accuracy = tf.keras.metrics.Accuracy()

In [ ]:
model = keras.Sequential(
    [
        keras.layers.InputLayer(input_shape=(32, 32,3),batch_size=64),
     
        keras.layers.Conv2D(32, (3, 3), name="conv1", activation='relu',padding='same'),
        keras.layers.BatchNormalization(name="bn1"),  
        keras.layers.Conv2D(32, (3, 3),name="conv2", activation='relu', padding='same'),
        keras.layers.BatchNormalization(name="bn2"),
        keras.layers.MaxPooling2D(pool_size=(2, 2),name="mp1"),
        #keras.layers.Dropout(0.2),
        keras.layers.Conv2D(64, (3, 3),name="conv3", activation='relu', padding='same'),
        keras.layers.BatchNormalization(name="bn3"),
        keras.layers.Conv2D(64, (3, 3),name="conv4", activation='relu', padding='same'),
        keras.layers.BatchNormalization(name="bn4"),
        keras.layers.MaxPooling2D(pool_size=(2, 2),name="mp2"),
        #keras.layers.Dropout(0.2),
        keras.layers.Conv2D(128, (3, 3),name="conv5", activation='relu', padding='same'),
        keras.layers.BatchNormalization(name="bn5"),
        keras.layers.Conv2D(128, (3, 3),name="conv6", activation='relu', padding='same'),
        keras.layers.BatchNormalization(name="bn6"),
        keras.layers.MaxPooling2D(pool_size=(2, 2),name="mp3"),
        #keras.layers.Dropout(0.2),
     
        keras.layers.Flatten(name = "flatten"),
        keras.layers.Dropout(0.2, name="dp1"),
        keras.layers.Dense(1024, name="dense1", activation='relu'),
        keras.layers.Dropout(0.2),
        #keras.layers.Dense(256, name="dense2", activation='relu'),
        #keras.layers.Dropout(0.4),
        #keras.layers.Dense(128, name="dense3", activation='relu'),
        #keras.layers.BatchNormalization(name="bn7"),
        #keras.layers.Dropout(0.4),
        keras.layers.Dense(10, name="dense4", activation='softmax')
    ],
    name="modelo-0.0",
)

for layer in model.trainable_variables:
    #print(layer.name)
    if 'bias' in layer.name:
        new_bias = tf.cast(tf.where(tf.abs(layer) >= 0, 0.1, 0.1), tf.float32)
        layer.assign(new_bias)

model.summary()

Model: "modelo-0.0"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (Conv2D)              (64, 32, 32, 32)          896       
                                                                 
 bn1 (BatchNormalization)    (64, 32, 32, 32)          128       
                                                                 
 conv2 (Conv2D)              (64, 32, 32, 32)          9248      
                                                                 
 bn2 (BatchNormalization)    (64, 32, 32, 32)          128       
                                                                 
 mp1 (MaxPooling2D)          (64, 16, 16, 32)          0         
                                                                 
 conv3 (Conv2D)              (64, 16, 16, 64)          18496     
                                                                 
 bn3 (BatchNormalization)    (64, 16, 16, 64)          2

In [ ]:
epochs = 50
alpha = 0.01
n_bits = 32
learning_rate = 0.01
np.set_printoptions(threshold=np.inf)

model_sparsity = np.array([])
model_train_loss = np.array([])
model_train_acc = np.array([])
model_test_loss = np.array([])
model_test_acc = np.array([])
sparsity = 0


for epoch in range(epochs):
    # Iterate over the batches of the dataset.
    loss_batch = np.array([])
    loss_test_batch = np.array([])
    for step, (x_batch_train, y_batch_train) in enumerate(train_ds):
      n_zeros = 0
      size = 0
      #pruning
      if alpha > 0:
        bk = []
        for layer_weights in model.trainable_variables:
          if 'bn' in layer_weights.name:
              bk.append(-1)
          else:
              #flatten weights
              f_weights = tf.reshape(layer_weights,[-1])
              #get standard deviation of each layer
              lim = alpha*tf.math.reduce_std(f_weights)
              bk.append(lim)
              #create a mask
              mask = tf.cast(tf.where(tf.abs(layer_weights)>lim,1,0), tf.float32)
              #assign pruned weights to the layer
              layer_weights.assign(tf.math.multiply(layer_weights,mask))
              #check sparsity
              flat_array = np.array((tf.reshape(mask,[-1])))
              n_zeros += np.count_nonzero(np.array(flat_array) == 0)
              size += flat_array.shape[0]
              sparsity = n_zeros*100/size
      else:
        bk = [0] * len(model.trainable_weights)

      #Cópia do modelo
      if n_bits > 0 and alpha > 0:
          model_copy = keras.models.clone_model(model)
          model_copy.set_weights(model.get_weights())
      
      #Quantização
      if n_bits > 0 and alpha > 0:
          for i, layer_weights in enumerate(model.trainable_variables):
              if 'bn' in layer_weights.name:
                  pass
              else:
                  qk_line = (tf.reduce_max(tf.math.abs(layer_weights)) - bk[i]) / (2 ** (n_bits - 1) - 1)
                  ck = tf.math.round(layer_weights / qk_line) * qk_line
                  layer_weights.assign(ck)

      with tf.GradientTape() as tape:
        pred = model(x_batch_train, training=True)
        loss = loss_fn(y_batch_train, pred)
        
      grads = tape.gradient(loss, model.trainable_weights)

      if n_bits > 0 and alpha > 0:
          for i, (layer_weights, copied_weights) in enumerate(zip(model.trainable_variables, model_copy.trainable_variables)):
              grads[i] = grads[i] * learning_rate
              # WEIGHT UPDATE
              layer_weights.assign(tf.math.subtract(copied_weights, grads[i]))
      else:
          for i, layer_weights in enumerate(model.trainable_variables):
              grads[i] = grads[i] * learning_rate
              #WEIGHT UPDATE
              layer_weights.assign(tf.math.subtract(layer_weights, grads[i]))
          #optimizer.apply_gradients(zip(grads, model.trainable_weights))
      
      #optimizer.apply_gradients(zip(grads, model.trainable_weights))

      predictions = tf.argmax(pred, axis=1, output_type=tf.int32)
      acc = train_accuracy(y_batch_train, predictions)

      #keep track of loss by batch
      loss_batch = np.append(loss_batch, loss)

    #mean of loss by epoch
    model_train_loss = np.append(model_train_loss, np.mean(loss_batch))
    #train accuracy by epoch
    model_train_acc = np.append(model_train_acc, acc*100)
    #sparsity by epoch
    model_sparsity = np.append(model_sparsity, sparsity)

    if alpha > 0:
        bk = []
        for layer_weights in model.trainable_variables:
            if 'bn' in layer_weights.name:
                bk.append(-1)
            else:
                #flatten weights
                f_weights = tf.reshape(layer_weights,[-1])
                #get standard deviation of each layer
                lim = alpha*tf.math.reduce_std(f_weights)
                bk.append(lim)
                #create a mask
                mask = tf.cast(tf.where(tf.abs(layer_weights)>lim,1,0), tf.float32)
                #assign pruned weights to the layer
                layer_weights.assign(tf.math.multiply(layer_weights,mask))
                #check sparsity
                flat_array = np.array((tf.reshape(mask,[-1])))
                n_zeros += np.count_nonzero(np.array(flat_array) == 0)
                size += flat_array.shape[0]
                sparsity = n_zeros*100/size
    else:
        bk = [0] * len(model.trainable_weights)

    #Quantização
    if n_bits > 0 and alpha > 0:
        for i, layer_weights in enumerate(model.trainable_variables):
            if 'bn' in layer_weights.name:
                pass
            else:
                qk_line = (tf.reduce_max(tf.math.abs(layer_weights)) - bk[i]) / (2 ** (n_bits - 1) - 1)
                ck = tf.math.round(layer_weights / qk_line) * qk_line
                layer_weights.assign(ck)
    
    bk.clear()
    
    #Test
    for step, (x_batch_test, y_batch_test) in enumerate(test_ds):
      test_pred = model(x_batch_test, training=False)
      test_loss = loss_fn(y_batch_test,test_pred)
      test_prediction = tf.argmax(test_pred, axis=1, output_type=tf.int32)
      test_acc = test_accuracy(y_batch_test, test_prediction)
      
      loss_test_batch = np.append(loss_batch,test_loss)

    model_test_acc = np.append(model_test_acc, test_acc*100)
    model_test_loss = np.append(model_test_loss,np.mean(loss_test_batch))
    
    print("Epoch {}/{} \t Loss = {:.3f} \t Train Acc = {:.3f}% \t Sparsity = {:.3f}% \t Test Acc = {:.3f}%".format(epoch+1,epochs,float(loss),float(acc*100),sparsity,float(test_acc*100)))

Epoch 1/50 	 Loss = 1.266 	 Train Acc = 47.708% 	 Sparsity = 2.674% 	 Test Acc = 54.520%
Epoch 2/50 	 Loss = 0.798 	 Train Acc = 55.510% 	 Sparsity = 3.320% 	 Test Acc = 59.035%
Epoch 3/50 	 Loss = 0.757 	 Train Acc = 60.227% 	 Sparsity = 3.751% 	 Test Acc = 62.723%
Epoch 4/50 	 Loss = 0.368 	 Train Acc = 63.459% 	 Sparsity = 4.120% 	 Test Acc = 64.707%
Epoch 5/50 	 Loss = 0.593 	 Train Acc = 65.964% 	 Sparsity = 4.411% 	 Test Acc = 66.530%
Epoch 6/50 	 Loss = 0.709 	 Train Acc = 68.023% 	 Sparsity = 4.674% 	 Test Acc = 67.340%
Epoch 7/50 	 Loss = 0.528 	 Train Acc = 69.719% 	 Sparsity = 4.943% 	 Test Acc = 68.139%
Epoch 8/50 	 Loss = 0.676 	 Train Acc = 71.188% 	 Sparsity = 5.162% 	 Test Acc = 69.079%
Epoch 9/50 	 Loss = 0.555 	 Train Acc = 72.511% 	 Sparsity = 5.436% 	 Test Acc = 69.639%
Epoch 10/50 	 Loss = 0.409 	 Train Acc = 73.725% 	 Sparsity = 5.680% 	 Test Acc = 70.383%
Epoch 11/50 	 Loss = 0.572 	 Train Acc = 74.832% 	 Sparsity = 5.825% 	 Test Acc = 70.962%
Epoch 12/50 	 Loss 

In [ ]:
l = len(model.trainable_weights)
for i in range(l):
  a = tf.reshape(model.trainable_weights[i],[-1])
  b = a.numpy()
  #print(a)
  #plt.ylim(0,300)
  plt.title(str(i))
  plt.hist(b,200)
  plt.show()

In [ ]:
plt.title("test acc x train acc")
plt.plot(model_train_acc)
plt.plot(model_test_acc)
plt.show()

In [ ]:
plt.title("train loss x test loss")
plt.plot(model_train_loss)
plt.plot(model_test_loss)
plt.show()

In [ ]:
plt.title("Sparsity")
plt.plot(model_sparsity)
plt.show()

In [ ]:
%matplotlib inline
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    figure(figsize=(10, 7), dpi=80)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(10)
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm[:,0])

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, str(cm[i, j]*100/1000) + "%",
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Classe real')
    plt.xlabel('Classe predita')

In [ ]:
test_accuracy = tf.keras.metrics.Accuracy()
logits = model(x_test, training=False)
prediction = tf.argmax(logits, axis=1, output_type=tf.int32)
test_accuracy(prediction, y_test)
print("Test set accuracy: {:.3%}".format(test_accuracy.result()))

In [ ]:
classes = ["avião","carro","pássaro","gato","veado","cachorro","sapo","cavalo","navio","caminhão"]
cm = confusion_matrix(y_true=y_test, y_pred=prediction)
plot_confusion_matrix(cm=cm, classes=classes, title='Matriz de confusão')